IA & Data science (LU3IN0226) -- 2024-2025
--------
*&copy; Equipe pédagogique: Christophe Marsala, Olivier Schwander, Jean-Noël Vittaut.*


# TD-TME 6 : Apprentissage pour le texte

<font size="+1" color="RED"><b>[Q]</b></font> **Indiquer dans la boîte ci-dessous vos noms et prénoms :**

Thibaut MARCQ

<font color="RED" size="+1"><b>[Q]</b></font> **Renommer ce fichier ipython**

Tout en haut de cette page, cliquer sur <tt>tme-05</tt> et rajouter à la suite de <tt>tme-05</tt> les noms des membres du binômes séparés par un tiret.

<font color="RED" size="+1">IMPORTANT: soumission de votre fichier final</font>

**Nom à donner au fichier à poster** : *tme-05-Nom1_Nom2.ipynb* 
- *Nom1* et *Nom2* : noms des membres du binôme
- ne pas compresser ou faire une archive: il faut rendre le fichier ipython tel quel, éventuellement, si vous avez d'autres fichiers vous les rendez séparément.

**Echancier pour la soumission de votre compte-rendu:**
- le compte-rendu d'une séance doit être remis obligatoirement <font color="RED">avant la séance suivante</font>.

**Le compte-rendu est soumis sur la page Moodle.**

In [1]:
# - - - - - - - - - - - - - - - - - -
# imports utiles
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib as mtpl
%matplotlib inline  

import math
import time
import sys

# Les instructions suivantes sont utiles pour recharger automatiquement 
# le code modifié dans les librairies externes
%load_ext autoreload
%autoreload 2

# - - - - - - - - - - - - - - - - - -
# Information sur l'environnent utilisé ici:
print("Version python et des librairies:")
print("\tPython ",sys.version)
print("\tpandas: ",pd.__version__)
print("\tnumpy: ",np.__version__)
print("\tmatplotlib: ",mtpl.__version__)

/tmp/ipykernel_6273/1189054792.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


Version python et des librairies:
	Python  3.11.2 (main, Nov 30 2024, 21:22:50) [GCC 12.2.0]
	pandas:  2.2.0
	numpy:  1.24.2
	matplotlib:  3.8.2


In [2]:
# Importation de votre librairie iads:
# La ligne suivante permet de préciser le chemin d'accès à la librairie iads
import sys
sys.path.append('../')   # iads doit être dans le répertoire père du répertoire courant !

# Importation de la librairie iads
import iads as iads

# importation de Classifiers
from iads import Classifiers as classif

# importation de utils
from iads import utils as ut

# importation de evaluation
from iads import evaluation as ev



# Objectifs de ce TME

<div class="alert alert-block alert-warning">
Ce TME a pour but d'implémenter des fonctions pour traiter un corpus textuel et un nouvel algorithme d'apprentissage vu dans le cours 6. 

Pour expérimenter, on utilise la base `SMS spam Collection` qui contient 5572 messages associés à 2 labels: `spam` et `ham`. 
</div>

In [3]:
# Chargement du dataset

df_spam = pd.read_csv('data/spam.csv', sep='\t', encoding = 'latin1')
df_spam

,label,message
0,ham,Go until jurong point crazy.. Available only ...
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,Nah I don't think he goes to usf he lives aro...
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will Ì_ b going to esplanade fr home?
5569,ham,Pity * was in mood for that. So...any other s...
5570,ham,The guy did some bitching but I acted like i'd...


In [4]:
# Plus petit échantillon de données (10% spam/ham)

spam = df_spam[df_spam['label']== 'spam']
ham = df_spam[df_spam['label']== 'ham']
idx_spam = int(0.1*len(spam))
idx_ham = int(0.1*len(ham))

sample_spam = spam.sample(n=idx_spam)
sample_ham = ham.sample(n=idx_ham)

df_sample = pd.concat([sample_spam, sample_ham])

df_sample

,label,message
5190,spam,Our records indicate u maybe entitled to 5000 ...
737,spam,Hi. Customer Loyalty Offer:The NEW Nokia6650 M...
3585,spam,I am hot n horny and willing I live local to y...
4154,spam,Want a new Video Phone? 750 anytime any networ...
2266,spam,<Forwarded from 88877>FREE entry into our å£25...
...,...,...
1666,ham,Guess who spent all last night phasing in and ...
840,ham,I luv u soo much u donåÕt understand how speci...
5095,ham,K.k.this month kotees birthday know?
5454,ham,Im just wondering what your doing right now?


In [5]:
# Valeurs du label:
df_spam['label'].unique()

array(['ham', 'spam'], dtype=object)

<font color="RED" size="+1"><b>[Q]</b></font> En utilisant `value_counts` (voir la doc de la librairie pandas), afficher le nombre d'exemples de chaque classe dans la base.

In [6]:
pd.value_counts(df_spam['label'])

/tmp/ipykernel_6273/3845764354.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(df_spam['label'])


label
ham     4825
spam     747
Name: count, dtype: int64

## Prétraitements

Pour traiter les données textuelles (colonne `message` du dataset), on utilise les fonctions de la librairie `string`.

In [7]:
import string

print("Caractères de ponctuations : ", string.punctuation)

print("Mise en minuscules (pour homogénéiser l'écriture : ", "May the Force be with you!".lower())

print("Découper une phrase avec espace (retourne une liste):", "LU3IN026 est l'UE d'IA et Sciences des données.".split())

print("Découper une phrase avec apostrophe (retourne une liste):", "LU3IN026 est l'UE d'IA et Sciences des données.".split("'"))



Caractères de ponctuations :  !"#$%&'()*+,-./:;<=>?@[\]^_`{|}~
Mise en minuscules (pour homogénéiser l'écriture :  may the force be with you!
Découper une phrase avec espace (retourne une liste): ['LU3IN026', 'est', "l'UE", "d'IA", 'et', 'Sciences', 'des', 'données.']
Découper une phrase avec apostrophe (retourne une liste): ['LU3IN026 est l', 'UE d', 'IA et Sciences des données.']


In [8]:
# Avec le dataframe du dataset:
df_spam['message'][1].split()

['Ok', 'lar...', 'Joking', 'wif', 'u', 'oni...']

In [9]:
# Certains éléments d'une phrase ne sont pas utiles pour le traitement, par exemple en anglais:
mots_inutiles = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

<font color="RED" size="+1"><b>[Q]</b></font> Ecrire la fonction `nettoyage` qui prend une chaîne de caractères et rend la chaîne après nettoyage: 1) mise en minuscules 2) remplacement des caractères de ponctuation par un espace (SAUF l'apostrophe qui doit rester car elle va être utilisée pour les mots inutiles).


In [10]:
def nettoyage(message : str):
    """
    input : 
        -chaine de caractères
    returns : 
        - chaine mise en minuscule, ponctuation mise en espaces (sauf apostrophe)
    """

    res = message.lower()
    for lettre in res :
        if lettre in string.punctuation:
            if lettre!="'":
                res = res.replace(lettre, ' ')
    return res
    

In [11]:
nettoyage("LU3IN026 est l'UE d'IA et Sciences des données.")

"lu3in026 est l'ue d'ia et sciences des données "

In [12]:
nettoyage(df_spam['message'][1])

'ok lar    joking wif u oni   '

In [13]:
nettoyage(df_spam['message'][1]).split()

['ok', 'lar', 'joking', 'wif', 'u', 'oni']

<font color="RED" size="+1"><b>[Q]</b></font> Ecrire la fonction `text2vect` qui prend une chaîne de caractères ainsi qu'une liste de mots inutiles et rend la liste composée par les mots de cette chaîne obtenus, après son nettoyage et après avoir enlevé les mots inutiles.



In [14]:
def text2vect(message:str, inutiles:str):
    """
    input: chaine de caractères
    output: chaine sans les mots de inutiles
    """
    res = []
    cleaned = nettoyage(message)
    for mot in cleaned.split() : 
        if mot not in inutiles :
            res.append(mot)
    return res

In [15]:
text2vect("May the Force be with you!",mots_inutiles)

['may', 'force']

In [16]:
text2vect(df_spam['message'][1],mots_inutiles)

['ok', 'lar', 'joking', 'wif', 'u', 'oni']

In [17]:
text2vect("You shan't not pass!",mots_inutiles)

['pass']

<font color="RED" size="+1"><b>[Q]</b></font> Ajouter une nouvelle colonne de nom `les_mots` au dataframe `df_spam` pour laquelle chaque ligne contient le résultat de l'application de `text2vect` sur le message de l'exemple correspondant.


In [18]:
# A COMPLETER
les_mots = []
for message in df_spam['message']:
    les_mots.append(text2vect(message, mots_inutiles))

df_spam['les_mots'] = les_mots
# --------------------- Affichage du résultat:
df_spam

,label,message,les_mots
0,ham,Go until jurong point crazy.. Available only ...,"[go, jurong, point, crazy, available, bugis, n..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, u, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, early, hor, u, c, already, say]"
4,ham,Nah I don't think he goes to usf he lives aro...,"[nah, think, goes, usf, lives, around, though]"
...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,"[2nd, time, tried, 2, contact, u, u, å£750, po..."
5568,ham,Will Ì_ b going to esplanade fr home?,"[ì, b, going, esplanade, fr, home]"
5569,ham,Pity * was in mood for that. So...any other s...,"[pity, mood, suggestions]"
5570,ham,The guy did some bitching but I acted like i'd...,"[guy, bitching, acted, like, i'd, interested, ..."


<font color="RED" size="+1"><b>[Q]</b></font> Construire la liste de tous les mots présents dans tous les vecteurs du dataframe, chaque mot doit n'apparaître qu'une seule fois dans la liste résultat. Cette liste sera stockée dans la variable `index_mots` (et on l'appelle par la suite "index de mots").


In [49]:
index_mots = []
for line in list(df_spam['les_mots']):
    for word in line:
        if word not in index_mots:
            index_mots.append(word)
# df_spam['les_mots'].unique()

index_mots = np.array(index_mots)

In [52]:
index_mots = set()
for line in list(df_spam['les_mots']):
    for word in line:
        index_mots.add(word)
# df_spam['les_mots'].unique()

index_mots = list(index_mots)

In [53]:
print("Nombre de mots trouvés: ", len(index_mots))
print("Les 10 premiers :", index_mots[0:10])


Nombre de mots trouvés:  8840
Les 10 premiers : ['minmoremobsemspobox45po139wa', 'prin', 'miserable', 'wall', 'others', 'mee', '09050000301', 'ears', 'photoshop', 'practicing']


Chaque message va maintenant être représenté comme un vecteur de valeurs 0 ou 1: ce vecteur a autant de colonne qu'il y a de mots dans `index_mots`. Ce vecteur est construit ainsi: pour un exemple $i$, la valeur de la colonne $j$ du vecteur vaudra 1 si la liste de mots de l'exemple $i$ contient le mot en position $j$ dans `index_mots`, ou 0 dans le cas contraire.

<font color="RED" size="+1"><b>[Q]</b></font> Ecrire la fonction `df2array` qui prend un dataframe `df` contenant la colonne `les_mots` ainsi qu'un index de mots et rend le `np.array` correspondant aux vecteurs de valeurs représentant les exemples de `df`.


In [22]:
def df2array(df, index_mots):
    """
    input: 
        -df: dataframe contenant la colonne 'les_mots' (liste de mots nettoyés et filtrés)
        -index: index de tous les mots de la bdd
    returns: 
        -nparray correspondant aux vecteurs des mots présents dans la bdd (1 si le mot est présent, 0 sinon
    """
    
    res = np.zeros((df.shape[0], len(index_mots)))
   
    for id_line in range(df['les_mots'].shape[0]):
        for word in df['les_mots'][id_line]:
            id_word = index_mots.index(word) #on cherche l'indice du mot dans l'index
            res[id_line, id_word] = 1
            
    return res

In [23]:
# Attention: le calcul peut prendre un certain temps
mat_spam = df2array(df_spam,index_mots)

In [24]:
mat_spam.shape

(5572, 8840)

In [25]:
print("Message 0:", df_spam['les_mots'][0])

print("positions non nulles dans le vecteur d'index:")

for i in range(0, len(mat_spam[0])):
    if mat_spam[0][i] == 1:
        print("colonne ",i," : ", index_mots[i])
    

Message 0: ['go', 'jurong', 'point', 'crazy', 'available', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amore', 'wat']
positions non nulles dans le vecteur d'index:
colonne  222  :  wat
colonne  366  :  e
colonne  631  :  got
colonne  1234  :  n
colonne  1238  :  crazy
colonne  1942  :  point
colonne  1951  :  buffet
colonne  3404  :  la
colonne  4809  :  amore
colonne  5092  :  cine
colonne  5178  :  bugis
colonne  6693  :  great
colonne  7174  :  world
colonne  8148  :  jurong
colonne  8229  :  go
colonne  8230  :  available


In [26]:
# maintenant, il est possible d'extraire les vecteurs correspondant à un label donné, par exemple:
len(mat_spam[df_spam['label']=='spam'])

747

<font color="RED" size="+1"><b>[Q]</b></font> Construire un dictionnaire qui donne, pour chaque label, la fréquence, colonne par colonne, des exemples qui possèdent le label.


In [80]:
les_labels = df_spam['label'].unique()

frequences = dict()
for l in les_labels:
    count = mat_spam[df_spam['label'] == l].sum(axis=0)  # somme de toutes les colonnes = count des mots
    size_count = len(mat_spam[df_spam['label'] == l])
    frequences[l] = np.array([0.0] * len(index_mots))
    for id_element in range(len(count)):
        frequences[l][id_element] = count[id_element] / size_count

    
# ########################################
# Affichage des valeurs non nulles:
for l in frequences:
    print(l)
    nb = 0
    for j in range(0, len(frequences[l])):
        if frequences[l][j] != 0:
            if nb < 10:
                print(f'\t {index_mots[j]}:\t {frequences[l][j]:0.6f}')
            nb += 1
    print("Seuls les 10 premiers non nuls sont affichés.")

ham
	 prin:	 0.000207
	 miserable:	 0.000415
	 wall:	 0.000207
	 others:	 0.001865
	 mee:	 0.000207
	 ears:	 0.000207
	 photoshop:	 0.000207
	 practicing:	 0.000415
	 abj:	 0.000415
	 awake:	 0.001451
Seuls les 10 premiers non nuls sont affichés.
spam
	 minmoremobsemspobox45po139wa:	 0.002677
	 09050000301:	 0.001339
	 helen:	 0.001339
	 50gbp:	 0.004016
	 87066:	 0.017403
	 cheaper:	 0.002677
	 award:	 0.025435
	 09066364349:	 0.001339
	 tonexs:	 0.001339
	 news:	 0.013387
Seuls les 10 premiers non nuls sont affichés.


In [81]:
print(frequences['ham'])

[0.         0.00020725 0.00041451 ... 0.         0.         0.00020725]


In [82]:
#index_mots.index('tried')

print(f" {frequences['spam'][index_mots.index('jurong')]:0.6f}")

 0.000000


## Classification de textes

<div class="alert alert-block alert-warning">
On considère deux variables $X$ et $Y$ :
    <ul>
        <li>$X$ est un message</li>
        <li>$Y$ est le label d'un message et peut prendre 2 valeurs : <code>'ham'</code> et <code>'spam'</code></li>
    </ul>

Avec les fonctions précédentes on peut représenter les messages d'un corpus de documents sous la forme d'un vecteur $X \in \{0, 1\}^p$ où $p$ est le nombre total de mots de l'index. Le $i$ème message est représenté par le vecteur ${\bf x}_i = [x_{i1} \dots x_{ip}]$, où $x_{ij}$ vaut 1 si le $j$ème mot de l'index est présent dans le message $i$, et 0 sinon.
    
Comme vu en cours, pour un classifieur bayésien, nous devons estimer, à partir de la base <code>df_spam</code>, les probabilités $p(ham)$, $p(spam)$, $p(X |ham)$ et $p(X | spam)$.

Les 2 premières sont simples à calculer : on compte le nombre de fois où le label apparaît parmi les exemples.

Pour un label $Y$ et un exemple ${\bf x}$, le calcul de $p({\bf x} | Y)$ se fait en utilisant les probabilités $p(mot | Y)$ de tous les mots qui composent ${\bf x}$ (et qui sont des mots de l'index des mots). $p(mot | Y)$ est la probabilité que le mot <code>mot</code> apparaisse dans un message sachant que ce message appartient à la classe $Y$.
    
On pose ainsi

$$ p({\bf x} | Y) = \prod_{mot \in index\_mots} p(mot | Y)^{x_{mot}} \left(1 - p(mot | Y)\right)^{1 - x_{mot}} $$

où $x_{mot}$ correspond à $1$ ou $0$ selon que le `mot` apparaît dans ${\bf x}$ ou pas. Ce terme permet de retenir soit la probabilité $p(mot | Y)$ si le mot est dans ${\bf x}$, soit la probabilité qu'il n'y soit pas ($1-p(mot |Y)$).
    
Une fois que $p({\bf x} | Y)$ est calculée, on peut estimer $p(Y|X)$ grâce au théorème de Bayes (cf. cours 6):

$$p(Y|{\bf x}) = p(Y) p({\bf x} | Y)$$

avec $Y$ qui vaut soit 'ham', soit 'spam'.    

    
Une fois $p(Y|{{\bf x}})$ calculée pour chaque valeur de label, pour prédire le label de ${\bf x}$ on choisit le label qui possède la plus forte probabilité.
</div>

In [83]:
# calculer p(ham) et p(spam) dans df_spam:

for l in ['ham', 'spam']:
    proba = len(df_spam[df_spam['label']==l]) / len(df_spam)

    print(f'p({l:4}) = {proba:0.4f}')


p(ham ) = 0.8659
p(spam) = 0.1341


<font color="RED" size="+1"><b>[Q]</b></font> Ecrire la fonction `proba_mot` qui étant donné un mot, un label, une liste de mots, et un dictionnaire comme `frequences` (compatible avec la liste de mots) rend $p(mot| label)$ la probabilité du mot d'appartenir au label donné.


In [84]:
def proba_mot(mot:str, label:str, index_mots:list, frequences:dict):
    """
    input: 
        -label: 'ham' ou 'spam'
        -mot: mot dont on veut la probabilité
        -frequences: dictionnaire des fréquences des mots
        -index_mots: liste des mots de la bdd
    returns: 
        -probabilité que mot appartienne à label
    """
    if mot in index_mots and label in frequences:
        return frequences[label][index_mots.index(mot)]
    else:
        return 0

In [85]:
# probabilité d'un mot qui n'est pas dans l'index:
proba_mot("toto", "ham", index_mots, frequences)

0

In [67]:
# probabilité d'un mot de l'index pour un label qui n'existe pas:
proba_mot("visit", "cookie", index_mots, frequences)

0

In [86]:
# probabilité d'un mot de l'index pour un label qui existe:
proba_mot("visit", "ham", index_mots, frequences)

0.0018652849740932642

In [87]:
# probabilité d'un mot de l'index pour un label qui existe:
proba_mot("visit", "spam", index_mots, frequences)

0.009370816599732263

<font color="RED" size="+1"><b>[Q]</b></font> Ecrire la fonction `proba_exemple` qui étant donné un exemple représenté sous la forme d'une liste de mots, un label, une liste de mots, et un dictionnaire comme `frequences` (compatible avec la liste de mots) rend $p(exemple|label)$ la probabilité de l'exemple d'appartenir au label.


In [91]:
def proba_exemple(exemple:list, label:str, index_mots:list, frequences:dict):
    """
    input: 
        -exemple: liste de mots
        -label: 'ham' ou 'spam'
        -frequences: dictionnaire des fréquences des mots
        -index_mots: liste des mots de la bdd
    returns: 
        -probabilité que exemple appartienne à label
    """
    res = 1
    for mot in index_mots:
        if mot in exemple:
            res *= proba_mot(mot, label, index_mots, frequences)
        else:
            res *= (1- proba_mot(mot, label, index_mots, frequences))
    return res

In [92]:
proba_exemple(df_spam['les_mots'][0], 'ham', index_mots,frequences)

8.276300074060088e-43

In [93]:
proba_exemple(df_spam['les_mots'][0], 'spam', index_mots,frequences)


0.0

In [95]:
proba_exemple(df_spam['les_mots'][4], 'ham', index_mots,frequences)

1.0889167353895188e-20

<font color="RED" size="+1"><b>[Q]</b></font> En utilisant toutes les fonctions écrites, le taux de bonne prédiction du classifieur bayésien naïf pour chaque valeur de label pour le dataset spam.

In [105]:
les_labels = df_spam['label'].unique()

# La variable suivante permettra de stocker les résultats
taux = dict()
for l in les_labels:
    taux[l] = dict()
    taux[l][True] = 0    # nombre de bien classés
    taux[l][False] = 0   # nombre de mal classés
    
# ------------------------ 
for id_line in range(0, 10):
    val_ham = proba_exemple(df_spam['les_mots'][id_line], 'ham', index_mots,frequences)
    val_spam = proba_exemple(df_spam['les_mots'][id_line], 'spam', index_mots,frequences)
    print(f"Message {id_line} : {val_ham, val_spam}")
    if val_ham > val_spam:
        if (df_spam['label'][id_line] == 'ham'):
            taux['ham'][True] += 1
        else :
            taux['ham'][False] += 1
    else:
        if (df_spam['label'][id_line] == 'spam'):
            taux['spam'][True] += 1
        else :
            taux['spam'][False] += 1


# ----------------------------------------------------
print("Résultats des 10 premiers classements : ", taux)
# le résultat total peut prendre un certain temps...

# Il reste à calculer les taux de bonne classification par label

Message 0 : (8.276300074060088e-43, 0.0)
Message 1 : (1.395051352449745e-16, 0.0)
Message 2 : (0.0, 3.02224083189476e-47)
Message 3 : (7.117071679822502e-18, 0.0)
Message 4 : (1.0889167353895188e-20, 0.0)
Message 5 : (0.0, 1.9575583785492967e-47)
Message 6 : (4.428429080981951e-22, 0.0)
Message 7 : (2.5403995004170723e-49, 0.0)
Message 8 : (0.0, 1.1824247324388976e-36)
Message 9 : (0.0, 1.5255714095456845e-29)
Résultats des 10 premiers classements :  {'ham': {True: 6, False: 0}, 'spam': {True: 4, False: 0}}


## Evaluation du classifieur Naive Bayes

Pour tout ce que l'on a fait jusque-là, on a travaillé sur tout le dataset, pour pouvoir l'évaluer il est nécessaire de le séparer en ensemble d'apprentissage et ensemble de test.

<font color="RED" size="+1"><b>[Q]</b></font> Découper aléatoirement `df_spam` en 2 parties égales, chacune contenant des exemples des 2 labels, avec la même distribution des labels dans chaque partie. 
Une des parties sert à apprendre l'index des mots et leurs fréquences.
L'autre partie n'est utilisée que pour calculer le taux de bonne classification par label.

Donner ensuite les taux de bonne classification par label pour l'ensemble de train et pour l'ensemble de test.

*Remarque*: certains mots de la partie de test pourront ne pas être présents dans l'index de mots car ils peuvent être absents de la partie d'apprentissage.

<font color="RED" size="+1"><b>[Q]</b></font> Mettre en place une approche par validation croisée pour évaluer le taux de bonne classification moyen de cette approche. 

<font color="RED" size="+1"><b>[Q]</b></font> Comparer les résultats obtenus (taux de bonne classification avec la validation croisée, temps de calcul) avec ceux obtenus avec l'application d'un classifieur par $k$ plus proches voisins. Pour cela, la `mat_spam` doit être utilisée comme description des données. 